In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import tensorflow as tf

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
%matplotlib inline

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
path = "./data/" 
train_df = pd.read_csv(path+'train.csv',index_col=0)
test_df = pd.read_csv(path+'test.csv',index_col=0)
sample_submission_df = pd.read_csv(path+'sample_submission.csv',index_col=0)
pd.options.display.max_columns = 30

## 전처리
 - 이상치 확인 및 처리

In [3]:
train_df.shape

(199991, 22)

In [4]:
train_df.describe()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z
count,199991.000000,1.999910e+05,199991.000000,199991.000000,199991.000000,199991.000000,1.999910e+05,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000,199991.000000
mean,360.830152,-6.750146e+00,18.675373,18.401235,18.043495,17.663526,1.084986e+01,19.072693,19.134483,18.183331,18.000882,21.837903,18.454136,18.481525,17.686617,17.699207,20.110991,18.544375,18.181544,17.692395,17.189281
std,225.305890,1.187678e+04,155.423024,127.128078,116.622194,123.735298,4.172116e+03,749.256162,90.049058,122.378972,145.862346,789.472333,154.376277,97.240448,145.730872,142.691880,122.299062,161.728183,133.984475,131.183416,133.685138
min,1.000000,-5.310802e+06,-40022.466071,-27184.795793,-26566.310827,-24878.828280,-1.864766e+06,-215882.917191,-21802.656144,-20208.516262,-26505.602101,-24463.431833,-25958.752324,-23948.588523,-40438.184078,-30070.729379,-26236.578659,-36902.402336,-36439.638493,-38969.416822,-26050.710196
25%,174.000000,1.965259e+01,18.701180,18.048572,17.747663,17.425523,1.994040e+01,18.902851,18.259352,17.903615,17.606148,19.247795,18.113933,17.479794,17.050294,16.804705,19.266214,18.076120,17.423425,16.977671,16.705774
50%,349.000000,2.087136e+01,19.904235,19.454492,19.043895,18.611799,2.104910e+01,20.069038,19.631419,19.188763,18.710967,20.366848,19.586559,19.182789,18.693370,18.174592,20.406840,19.547674,19.143156,18.641756,18.100997
75%,526.000000,2.216043e+01,21.150297,20.515936,20.073528,19.883760,2.233754e+01,21.385830,20.773911,20.331419,20.133179,21.797480,21.004397,20.457491,20.019112,19.807652,21.992898,20.962386,20.408140,19.968846,19.819554
max,1000.000000,1.877392e+04,3538.984910,3048.110913,4835.218639,9823.740407,4.870154e+03,248077.513380,12084.735440,8059.638535,18358.921741,298771.019041,12139.815877,7003.136546,9772.190537,17403.789263,14488.251976,10582.058590,12237.951703,4062.499371,7420.534172


In [5]:
def del_outlier(data, min=0, max=60):
    up_idx_t=()
    dw_idx_t=()
    train_light = data.iloc[:,2:]
    for i in range(len(train_light.columns)):
        col = train_light.columns[i]
        up_idx_t+=tuple(data[data[col]>max].index)
        dw_idx_t+=tuple(data[data[col]<min].index)
    del_idx = set(up_idx_t+dw_idx_t)
    
    data[~data.index.isin(del_idx)].describe()
    
    return data

In [6]:
train_df = del_outlier(train_df, min=-60, max=60)

In [7]:
train_df.columns, train_df.shape

(Index(['type', 'fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
        'psfMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i',
        'fiberMag_z', 'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i',
        'petroMag_z', 'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i',
        'modelMag_z'],
       dtype='object'), (199991, 22))

## DATA Setting

In [8]:
column_number = {}
number_columns = {}
for i, column in enumerate(sample_submission_df.columns):
    column_number[column] = i
    number_columns[i] = column
    
    
def to_number(x, dic):
    return dic[x]

train_df['type_num'] = train_df['type'].apply(lambda x: to_number(x, column_number))


In [9]:
def add_minus_feature(data,test = False):
    from itertools import combinations
    n = 0
    for count in range(5,21,5):
        s = 2
        if test == True :
            s = 1
        selected = data.columns[s:].values[n:count]
        mag = str.split(selected[0],'_')[0]
        for combi in list(combinations(selected,2)):
            name_1st = str.split(combi[0],'_')[1]
            name_2nd = str.split(combi[1],'_')[1]
            data[mag+"_"+name_1st+"-"+name_2nd] = data[combi[0]]-data[combi[1]]
        n=count
    return data

In [10]:
train_df = add_minus_feature(train_df)
test_df = add_minus_feature(test_df,True)

In [11]:
train_X = train_df.drop(columns=['type', 'type_num'], axis=1)
train_y = train_df['type_num']
test_X = test_df

In [12]:
train_X.head()

,fiberID,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,...,petroMag_g-i,petroMag_g-z,petroMag_r-i,petroMag_r-z,petroMag_i-z,modelMag_u-g,modelMag_u-r,modelMag_u-i,modelMag_u-z,modelMag_g-r,modelMag_g-i,modelMag_g-z,modelMag_r-i,modelMag_r-z,modelMag_i-z
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,601,23.198224,21.431953,21.314148,21.176553,21.171444,22.581309,21.644453,21.657571,21.387653,21.572827,22.504317,21.431636,21.478312,21.145409,...,0.286226,1.009190,0.332902,1.055866,0.722964,1.283708,1.385054,1.728637,1.601901,0.101347,0.444929,0.318194,0.343582,0.216847,-0.126735
1,788,21.431355,20.708104,20.678850,20.703420,20.473229,21.868797,21.029773,20.967054,20.937731,21.063646,21.360701,20.778968,20.889705,20.639812,...,0.139156,0.132308,0.249893,0.243045,-0.006847,0.734428,0.739030,0.799566,0.980641,0.004602,0.065138,0.246213,0.060537,0.241611,0.181074
2,427,17.851451,16.727898,16.679677,16.694640,16.641788,18.171890,17.033098,16.999682,17.095999,17.076449,17.867253,16.738784,16.688874,16.744210,...,-0.005426,-0.069222,-0.055336,-0.119132,-0.063796,1.120628,1.176814,1.157885,1.129134,0.056186,0.037257,0.008506,-0.018929,-0.047680,-0.028751
3,864,20.789900,20.040371,19.926909,19.843840,19.463270,21.039030,20.317165,20.217898,20.073852,19.794505,20.433907,19.993727,19.985531,19.750917,...,0.242810,0.538610,0.234614,0.530413,0.295800,0.769012,0.880913,1.012598,1.217856,0.111901,0.243586,0.448844,0.131685,0.336943,0.205258
4,612,26.454969,23.058767,21.471406,19.504961,18.389096,25.700632,23.629122,21.742750,19.861718,18.810375,25.859229,22.426929,21.673551,19.610012,...,2.816917,4.050788,2.063539,3.297411,1.233871,1.729059,3.401710,5.389723,6.501398,1.672651,3.660663,4.772338,1.988012,3.099688,1.111675


In [13]:
train_X.columns

Index(['fiberID', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i', 'psfMag_z',
       'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z',
       'petroMag_u', 'petroMag_g', 'petroMag_r', 'petroMag_i', 'petroMag_z',
       'modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z',
       'psfMag_u-g', 'psfMag_u-r', 'psfMag_u-i', 'psfMag_u-z', 'psfMag_g-r',
       'psfMag_g-i', 'psfMag_g-z', 'psfMag_r-i', 'psfMag_r-z', 'psfMag_i-z',
       'fiberMag_u-g', 'fiberMag_u-r', 'fiberMag_u-i', 'fiberMag_u-z',
       'fiberMag_g-r', 'fiberMag_g-i', 'fiberMag_g-z', 'fiberMag_r-i',
       'fiberMag_r-z', 'fiberMag_i-z', 'petroMag_u-g', 'petroMag_u-r',
       'petroMag_u-i', 'petroMag_u-z', 'petroMag_g-r', 'petroMag_g-i',
       'petroMag_g-z', 'petroMag_r-i', 'petroMag_r-z', 'petroMag_i-z',
       'modelMag_u-g', 'modelMag_u-r', 'modelMag_u-i', 'modelMag_u-z',
       'modelMag_g-r', 'modelMag_g-i', 'modelMag_g-z', 'modelMag_r-i',
       'modelMag_r-z', 'modelMag_i-z'],


In [14]:
## u-g, g-r, r-i, i-z 만 남기는게 좋지 않을까????
train_X.drop(['psfMag_u-r','psfMag_u-i','psfMag_u-z','psfMag_g-i','psfMag_g-z','psfMag_r-z',
             'fiberMag_u-r','fiberMag_u-i','fiberMag_u-z','fiberMag_g-i','fiberMag_g-z','fiberMag_r-z',
             'petroMag_u-r','petroMag_u-i','petroMag_u-z','petroMag_g-i','petroMag_g-z','petroMag_r-z',
             'modelMag_u-r','modelMag_u-i','modelMag_u-z','modelMag_g-i','modelMag_g-z','modelMag_r-z'] ,axis=1, inplace=True)

test_X.drop(['psfMag_u-r','psfMag_u-i','psfMag_u-z','psfMag_g-i','psfMag_g-z','psfMag_r-z',
             'fiberMag_u-r','fiberMag_u-i','fiberMag_u-z','fiberMag_g-i','fiberMag_g-z','fiberMag_r-z',
             'petroMag_u-r','petroMag_u-i','petroMag_u-z','petroMag_g-i','petroMag_g-z','petroMag_r-z',
         'modelMag_u-r','modelMag_u-i','modelMag_u-z','modelMag_g-i','modelMag_g-z','modelMag_r-z'] ,axis=1, inplace=True)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.3, random_state=42,stratify = train_y)

In [16]:
o_hot = OneHotEncoder()
y_train= o_hot.fit_transform(y_train.values.reshape(-1,1))
y_train = y_train.toarray()

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [17]:
o2_hot = OneHotEncoder()
y_test= o2_hot.fit_transform(y_test.values.reshape(-1,1))
y_test = y_test.toarray()

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [18]:
y_train.shape, y_test.shape

((139993, 19), (59998, 19))

In [19]:
X_train = X_train.values
X_train.shape

(139993, 37)

In [20]:
X_test = X_test.values
X_test.shape

(59998, 37)

In [ ]:

model.add(Dense(19, activation='softmax'))

from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

## 얼리스타핑
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50)

##최적모델 기억
mc = ModelCheckpoint('best_model.h5', monitor='loss', mode='min', save_best_only=True)

model.compile(loss='categorical_crossentropy',
             optimizer=Adam(0.001), metrics=['accuracy','categorical_crossentropy'])

history=model.fit(X_train, y_train, batch_size=100, epochs=5000, verbose=1,
                 callbacks=[es, mc])
### validation_data를 쓰면 test 데이터가 따로있을때 직접 넣을 수 있다.
model.evaluate(X_test, y_test)

Epoch 1/5000
139993/139993 [==============================] - 9s 63us/step - loss: 0.8704 - acc: 0.7445
Epoch 2/5000
139993/139993 [==============================] - 8s 56us/step - loss: 0.5941 - acc: 0.8104
Epoch 3/5000
139993/139993 [==============================] - 8s 57us/step - loss: 0.5537 - acc: 0.8175
Epoch 4/5000
139993/139993 [==============================] - 8s 56us/step - loss: 0.5318 - acc: 0.8235
Epoch 5/5000
139993/139993 [==============================] - 8s 58us/step - loss: 0.5159 - acc: 0.8278
Epoch 6/5000
139993/139993 [==============================] - 8s 58us/step - loss: 0.5095 - acc: 0.8290
Epoch 7/5000
139993/139993 [==============================] - 8s 59us/step - loss: 0.5083 - acc: 0.8293
Epoch 8/5000
139993/139993 [==============================] - 10s 70us/step - loss: 0.4947 - acc: 0.8335
Epoch 9/5000
139993/139993 [==============================] - 10s 69us/step - loss: 0.4850 - acc: 0.8368
Epoch 10/5000
139993/139993 [==============================] -

139993/139993 [==============================] - 7s 49us/step - loss: 0.4104 - acc: 0.8574
Epoch 79/5000
139993/139993 [==============================] - 6s 44us/step - loss: 0.4110 - acc: 0.8578
Epoch 80/5000
139993/139993 [==============================] - 7s 47us/step - loss: 0.4125 - acc: 0.8574
Epoch 81/5000
139993/139993 [==============================] - 6s 45us/step - loss: 0.4086 - acc: 0.8580
Epoch 82/5000
139993/139993 [==============================] - 7s 50us/step - loss: 0.4077 - acc: 0.8598
Epoch 83/5000
139993/139993 [==============================] - 8s 56us/step - loss: 0.4089 - acc: 0.8588
Epoch 84/5000
139993/139993 [==============================] - 8s 58us/step - loss: 0.4092 - acc: 0.8588
Epoch 85/5000
139993/139993 [==============================] - 8s 57us/step - loss: 0.4075 - acc: 0.8594
Epoch 86/5000
139993/139993 [==============================] - 7s 53us/step - loss: 0.4098 - acc: 0.8583
Epoch 87/5000
139993/139993 [==============================] - 7s 53u

139993/139993 [==============================] - 6s 40us/step - loss: 0.3997 - acc: 0.8612
Epoch 156/5000
139993/139993 [==============================] - 6s 40us/step - loss: 0.3984 - acc: 0.8623
Epoch 157/5000
139993/139993 [==============================] - 6s 40us/step - loss: 0.3990 - acc: 0.8619
Epoch 158/5000
139993/139993 [==============================] - 6s 46us/step - loss: 0.3986 - acc: 0.8616
Epoch 159/5000
139993/139993 [==============================] - ETA: 0s - loss: 0.3985 - acc: 0.862 - 6s 46us/step - loss: 0.3984 - acc: 0.8627
Epoch 160/5000
139993/139993 [==============================] - 6s 46us/step - loss: 0.3979 - acc: 0.8621
Epoch 161/5000
139993/139993 [==============================] - 6s 46us/step - loss: 0.3993 - acc: 0.8611
Epoch 162/5000
139993/139993 [==============================] - 6s 46us/step - loss: 0.3991 - acc: 0.8617
Epoch 163/5000
139993/139993 [==============================] - 7s 48us/step - loss: 0.3994 - acc: 0.8610
Epoch 164/5000
139993/1

139993/139993 [==============================] - 5s 39us/step - loss: 0.3932 - acc: 0.8638
Epoch 233/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3942 - acc: 0.8638
Epoch 234/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3960 - acc: 0.8629
Epoch 235/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3950 - acc: 0.8637
Epoch 236/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3952 - acc: 0.8629
Epoch 237/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3945 - acc: 0.8624
Epoch 238/5000
139993/139993 [==============================] - 6s 40us/step - loss: 0.3943 - acc: 0.8640
Epoch 239/5000
139993/139993 [==============================] - 6s 40us/step - loss: 0.3924 - acc: 0.8636
Epoch 240/5000
139993/139993 [==============================] - 6s 40us/step - loss: 0.3933 - acc: 0.8632
Epoch 241/5000
139993/139993 [==============================]

139993/139993 [==============================] - 6s 41us/step - loss: 0.3901 - acc: 0.8649
Epoch 310/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3918 - acc: 0.8640
Epoch 311/5000
139993/139993 [==============================] - 6s 42us/step - loss: 0.3907 - acc: 0.8640
Epoch 312/5000
139993/139993 [==============================] - 5s 38us/step - loss: 0.3885 - acc: 0.8645
Epoch 313/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3902 - acc: 0.8646
Epoch 314/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3921 - acc: 0.8639
Epoch 315/5000
139993/139993 [==============================] - 6s 40us/step - loss: 0.3911 - acc: 0.8638
Epoch 316/5000
139993/139993 [==============================] - 5s 39us/step - loss: 0.3892 - acc: 0.8648
Epoch 317/5000
139993/139993 [==============================] - 6s 39us/step - loss: 0.3904 - acc: 0.8635
Epoch 318/5000
139993/139993 [==============================]

...

139993/139993 [==============================] - 6s 41us/step - loss: 0.3817 - acc: 0.8667
Epoch 695/5000
139993/139993 [==============================] - 6s 41us/step - loss: 0.3822 - acc: 0.8676
Epoch 696/5000
139993/139993 [==============================] - 6s 41us/step - loss: 0.3810 - acc: 0.8676
Epoch 697/5000
139993/139993 [==============================] - 6s 41us/step - loss: 0.3812 - acc: 0.8675
Epoch 698/5000
139993/139993 [==============================] - 6s 42us/step - loss: 0.3827 - acc: 0.8673
Epoch 699/5000
139993/139993 [==============================] - 6s 42us/step - loss: 0.3824 - acc: 0.8668
Epoch 700/5000
139993/139993 [==============================] - 6s 42us/step - loss: 0.3824 - acc: 0.8681
Epoch 701/5000
139993/139993 [==============================] - 6s 42us/step - loss: 0.3816 - acc: 0.8677
Epoch 702/5000
139993/139993 [==============================] - 6s 41us/step - loss: 0.3820 - acc: 0.8671
Epoch 703/5000
139993/139993 [==============================]

139993/139993 [==============================] - 7s 48us/step - loss: 0.3804 - acc: 0.8682
Epoch 772/5000
139993/139993 [==============================] - 7s 53us/step - loss: 0.3812 - acc: 0.8663
Epoch 773/5000
139993/139993 [==============================] - 6s 44us/step - loss: 0.3803 - acc: 0.8680
Epoch 774/5000
139993/139993 [==============================] - 7s 51us/step - loss: 0.3814 - acc: 0.8674
Epoch 775/5000
139993/139993 [==============================] - 7s 50us/step - loss: 0.3789 - acc: 0.8682
Epoch 776/5000
139993/139993 [==============================] - 8s 56us/step - loss: 0.3806 - acc: 0.8682
Epoch 777/5000
139993/139993 [==============================] - 6s 42us/step - loss: 0.3806 - acc: 0.8675
Epoch 778/5000
139993/139993 [==============================] - 7s 53us/step - loss: 0.3803 - acc: 0.8673
Epoch 779/5000
139993/139993 [==============================] - 7s 49us/step - loss: 0.3806 - acc: 0.8675
Epoch 780/5000
139993/139993 [==============================]

139993/139993 [==============================] - 7s 47us/step - loss: 0.3812 - acc: 0.8673
Epoch 849/5000
139993/139993 [==============================] - 6s 46us/step - loss: 0.3808 - acc: 0.8686
Epoch 850/5000
139993/139993 [==============================] - 8s 58us/step - loss: 0.3789 - acc: 0.8690
Epoch 851/5000
139993/139993 [==============================] - 8s 59us/step - loss: 0.3795 - acc: 0.8676
Epoch 852/5000
139993/139993 [==============================] - 7s 49us/step - loss: 0.3790 - acc: 0.8685
Epoch 853/5000
139993/139993 [==============================] - 6s 46us/step - loss: 0.3785 - acc: 0.8689
Epoch 854/5000
139993/139993 [==============================] - 6s 43us/step - loss: 0.3806 - acc: 0.8680
Epoch 855/5000
139993/139993 [==============================] - 7s 48us/step - loss: 0.3804 - acc: 0.8673
Epoch 856/5000
139993/139993 [==============================] - 8s 57us/step - loss: 0.3801 - acc: 0.8678
Epoch 857/5000
139993/139993 [==============================]

139993/139993 [==============================] - 7s 53us/step - loss: 0.3784 - acc: 0.8681
Epoch 925/5000
139993/139993 [==============================] - 7s 50us/step - loss: 0.3796 - acc: 0.8681
Epoch 926/5000
139993/139993 [==============================] - 11s 79us/step - loss: 0.3803 - acc: 0.8680
Epoch 927/5000
139993/139993 [==============================] - 9s 67us/step - loss: 0.3798 - acc: 0.8677
Epoch 928/5000
139993/139993 [==============================] - 10s 73us/step - loss: 0.3782 - acc: 0.8679
Epoch 929/5000
139993/139993 [==============================] - 11s 77us/step - loss: 0.3782 - acc: 0.8689
Epoch 930/5000
139993/139993 [==============================] - 7s 52us/step - loss: 0.3794 - acc: 0.8685
Epoch 931/5000
139993/139993 [==============================] - 7s 49us/step - loss: 0.3802 - acc: 0.8677
Epoch 932/5000
139993/139993 [==============================] - 7s 49us/step - loss: 0.3790 - acc: 0.8686
Epoch 933/5000
139993/139993 [============================

Epoch 1001/5000
139993/139993 [==============================] - 7s 50us/step - loss: 0.3782 - acc: 0.8685
Epoch 1002/5000
139993/139993 [==============================] - 7s 53us/step - loss: 0.3773 - acc: 0.8683
Epoch 1003/5000
139993/139993 [==============================] - 7s 52us/step - loss: 0.3791 - acc: 0.8678
Epoch 1004/5000
139993/139993 [==============================] - 8s 57us/step - loss: 0.3784 - acc: 0.8682
Epoch 1005/5000
139993/139993 [==============================] - 6s 45us/step - loss: 0.3795 - acc: 0.8679
Epoch 1006/5000
139993/139993 [==============================] - 6s 42us/step - loss: 0.3782 - acc: 0.8689
Epoch 1007/5000
139993/139993 [==============================] - 6s 46us/step - loss: 0.3777 - acc: 0.8679
Epoch 1008/5000
139993/139993 [==============================] - 7s 49us/step - loss: 0.3784 - acc: 0.8681
Epoch 1009/5000
139993/139993 [==============================] - 7s 51us/step - loss: 0.3790 - acc: 0.8680
Epoch 1010/5000
139993/139993 [======

In [ ]:
##훈련 손실과 검증 손실 그래프 그리기



def plot_history(history, key='categorical_crossentropy'):
    plt.figure(figsize=(16,10))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])


plot_history(history)

In [ ]:
from sklearn.metrics import f1_score, recall_score
pred_y =  np.argmax(model.predict(test_x),axis=1)
y = np.argmax(test_y, axis=1)
print('f1_score:', f1_score(y, pred_y))
print('recall:', recall_score(y, pred_y))

In [36]:
print('f1_score:', f1_score(y, pred_y))
print('recall:', recall_score(y, pred_y))

f1_score: 0.45690376569037655
recall: 0.503690036900369


In [ ]:
real_y_pred = model.predict(test_x)

In [ ]:
submission = pd.DataFrame(data=real_y_pred, columns=sample_submission_df.columns, index=sample_submission_df.index)
submission.to_csv(f'submission_lightbgm_data_select_outlier_add_only({log_loss(y_test,train_y_pred,labels= list(set(y_test)))}).csv', index=True)